In [54]:
import imaplib
import clipboard
import email
import pandas as pd
import pyautogui
import re
import time
import smtplib
import yaml
from email.mime.text import MIMEText

In [ ]:
with open("credentials.yml") as f:
    content = f.read()
    
# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)


### data extraction from gmail

In [ ]:
ORG_EMAIL = "@gmail.com"
FROM_EMAIL = my_credentials['user']
FROM_PWD = my_credentials['password']
SMTP_SERVER = "imap.gmail.com"
SMTP_PORT = 993

def read_email():

    From=[]
    Subject=[]
    Body=[]
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(FROM_EMAIL,FROM_PWD)
        mail.select('inbox')

        data = mail.search(None, 'ALL')
        mail_ids = data[1]
        id_list = mail_ids[0].split()
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])

        # reading top 5 emails
        start = latest_email_id
        end = latest_email_id - 5

        for i in range(start,end, -1):
            data = mail.fetch(str(i), '(RFC822)' )
            for response_part in data:
                arr = response_part[0]
                if isinstance(arr, tuple):
                    msg = email.message_from_string(str(arr[1],'utf-8'))
                    email_subject = msg['subject']
                    email_from = msg['from']
                    # print('From : ' + email_from + '\n')
                    From.append(email_from)
                    # print('Subject : ' + email_subject + '\n')
                    Subject.append(email_subject)
                    if msg.is_multipart():
                        # If the email has multiple parts, iterate over them
                        for part in msg.walk():
                            # Check if the content type is text/plain
                            if part.get_content_type() == "text/plain":
                                # Get the email body and print it
                                email_body = part.get_payload(decode=True).decode()
                                # print('Message : ' + email_body + '\n')
                                Body.append(email_body)
                    else:
                        # If the email is not multipart, directly access the body
                        email_body = msg.get_payload(decode=True).decode()
                        # print('Message : ' + email_body + '\n')
                        Body.append(email_body)

    except Exception as e:
        traceback.print_exc()
        print(str(e))

    return From,Subject,Body

From,Subject,Body=read_email()


In [ ]:
def extract_data_from_text(text):

      # Extracting the topic
      topic_match = re.search(r"Topic - (.+)", text)
      topic = topic_match.group(1).strip() if topic_match else None

      # Extracting the time
      time_match = re.search(r"Time - (.+)", text)
      time = time_match.group(1).strip() if time_match else None

      # Extracting the type
      type_match = re.search(r"Type - (.+)", text)
      type = type_match.group(1).strip() if type_match else None

      # Extracting the host
      host_match = re.search(r"Host - (.+)", text)
      host = host_match.group(1).strip() if host_match else None

      # Extracting the date
      date_match = re.search(r"Date - (.+)", text)
      date = date_match.group(1).strip() if date_match else None


      # extracted information
      extracted_res={'topic':'','time':'','type':'','host':'','date':''}
      extracted_res['topic']=topic
      extracted_res['time']=time
      extracted_res['type']=type
      extracted_res['host']=host
      extracted_res['date']=date

      return extracted_res

In [ ]:
Subject_pattern = r'(?i)\b(?:meeting|conference|meet|talk|zoom meeting|zoom)\b'
ind =[]

for i in range(0,len(Subject)):

  matches = re.findall(Subject_pattern,Subject[i])
  if(len(matches)>=1):
    ind.append(i)

ind

In [ ]:
res = extract_data_from_text(Body[ind[0]]) # for latest one
res

### Zoom meeting creating


In [58]:
import pyautogui
import time
pyautogui.press('win')  # Press the Windows key
time.sleep(2)
pyautogui.typewrite('zoom') # Type 'notepad' to search for the Notepad application

time.sleep(2)
pyautogui.press('enter')  # Press Enter to open Notepad
time.sleep(2)  # Wait for Notepad to open

# clicking for scheduling
button7location = pyautogui.locateOnScreen('schedule.png') # returns (left, top, width, height) of matching region
button7location 
pyautogui.click(pyautogui.moveTo(button7location))
time.sleep(6)
# writting data
topic=res["topic"]
host=res["host"]
start_time=res["time"]
end_time=''

person_name="share"
def type_slowly(word):
    for character in word:
        pyautogui.write(character)
        time.sleep(0.1)

        
type_slowly(topic)

pyautogui.press('tab')
pyautogui.press('tab')
type_slowly(host)
time.sleep(1)

pyautogui.press('tab')
pyautogui.press('enter')
pyautogui.press('tab')
pyautogui.press('tab')

# type_slowly(start_date)
pyautogui.press('tab')
# pyautogui.press('tab')

type_slowly(start_time)
time.sleep(1)
pyautogui.press('tab')


# type_slowly(end_time)
time.sleep(1)
pyautogui.press('tab')


# type_slowly(end_time)
# pyautogui.press('tab')
# pyautogui.press('tab')

# after end_date
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('enter')
time.sleep(1)

# after generative
pyautogui.press('tab')
# type_slowly(passcode)
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('enter')

button7location = pyautogui.locateOnScreen('save.png') # returns (left, top, width, height) of matching region
button7location 
pyautogui.click(pyautogui.moveTo(button7location))

time.sleep(2)
button7location = pyautogui.locateOnScreen('copy_link.png') # returns (left, top, width, height) of matching region
button7location 
pyautogui.click(pyautogui.moveTo(button7location))
pyautogui.hotkey('alt', 'f4')
pyautogui.hotkey('alt', 'f4')
pyautogui.hotkey('ctrl', 'v')
link =clipboard.paste()
print(type(link))
print(link)


<class 'str'>
Baburam Yadav is inviting you to a scheduled Zoom meeting.

Topic: Talk on RPA
Time: Jun 25, 2023 12:40 PM India

Join Zoom Meeting
https://us05web.zoom.us/j/83664193626?pwd=S2ZFTDRYUHBHWDlNc3dOQzJqYmxiUT09

Meeting ID: 836 6419 3626
Passcode: 9iU7s4




### Code for sending mail

In [59]:


# Create a MIME text object with the email content
message = MIMEText(link)

# Set the sender and recipient addresses
FROM = my_credentials["user"]
message["From"] = FROM
message["To"] = my_credentials["to"]
message["Subject"] = "Zoom meeting"

# Connect to the SMTP server
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_password = my_credentials["password"]

with smtplib.SMTP(smtp_server, smtp_port) as server:
    # Start the secure connection if using TLS
    server.starttls()

    # Login to the SMTP server if authentication is required
    server.login(FROM, smtp_password)

    # Send the email
    server.send_message(message)

    # Close the connection
    print("message sent successfully")
    server.quit()


message sent successfully
